# Monitoring modèle

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

## 1. Infos

In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import AksWebservice
import azureml.core
import json

print('Version : ', azureml.core.VERSION)

Version :  1.0.83


In [2]:
import datetime
now = datetime.datetime.now()
print(now)

2020-02-26 11:56:30.443925


## 2. Worspace


In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

workshop-aml-2020
workshopaml2020RG
westeurope
70b8f39e-8863-49f7-b6ba-34a80799550c


## 3. Enregistrement du modèle de ML
Register an existing trained model, add descirption and tags.

In [4]:
#Register the model
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl", # this points to a local file
                       model_name = "sklearn_regression_model.pkl", # this is the name the model is registered as
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace = ws)

print(model.name, model.description, model.version)

Registering model sklearn_regression_model.pkl
sklearn_regression_model.pkl Ridge regression model to predict diabetes 4


## 4. Scoring File
Here is an example:
### a. In your init function add:
```python
print ("model initialized" + time.strftime("%H:%M:%S"))```

### b. In your run function add:
```python
print ("Prediction created" + time.strftime("%H:%M:%S"))```

In [5]:
%%writefile score.py
import os
import pickle
import json
import numpy 
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
import time

def init():
    global model
    #Print statement for appinsights custom traces:
    print ("model initialized" + time.strftime("%H:%M:%S"))
    
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
  

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        print ("Prediction created" + time.strftime("%H:%M:%S"))
        # you can return any datatype as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        print (error + time.strftime("%H:%M:%S"))
        return error

Overwriting score.py


## 5. myenv.yml
Please note that you must indicate azureml-defaults with verion >= 1.0.45 as a pip dependency, because it contains the functionality needed to host the model as a web service.

In [6]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'],
                                 pip_packages=['azureml-defaults'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

## 6. Create Inference Configuration

In [7]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

## Deploy to ACI (Optional)

from azureml.core.webservice import AciWebservice

aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "diabetes", 'type': "regression"}, 
                                               description = 'Predict diabetes using regression model',
                                               enable_app_insights = True)

from azureml.core.webservice import Webservice

aci_service_name = 'my-aci-service-4'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_deployment_config)
aci_service.wait_for_deployment(True)
print(aci_service.state)

%%time

test_sample = json.dumps({'data': [
    [1,28,13,45,54,6,57,8,8,10], 
    [101,9,8,37,6,45,4,3,2,41]
]})
test_sample = bytes(test_sample,encoding='utf8')

if aci_service.state == "Healthy":
    prediction = aci_service.run(input_data=test_sample)
    print(prediction)
else:
   raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aci_service.error)

## 7. Déploiement Azure Kubernetes Service (AKS)

> Le nom du service **aks** doit être disponible

In [9]:
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'aks-exemple12' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [10]:
%%time
aks_target.wait_for_completion(show_output = True)

Creating............................................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
CPU times: user 1.04 s, sys: 21 ms, total: 1.06 s
Wall time: 14min 41s


> Note : Prévoir 10 minutes de temps de traitement

### Voir dans le portail Azure le service AKS généré :

<img src="https://github.com/retkowsky/images/blob/master/akssnapshot.jpg?raw=true">

In [11]:
print("Etat AKS :", aks_target.provisioning_state)
print()
print("Erreur(s) AKS :",aks_target.provisioning_errors)

Etat AKS : Succeeded

Erreur(s) AKS : None


If you already have a cluster you can attach the service to it:

```python 
%%time
resource_id = '/subscriptions/<subscriptionid>/resourcegroups/<resourcegroupname>/providers/Microsoft.ContainerService/managedClusters/<aksservername>'
create_name= 'myaks4'
attach_config = AksCompute.attach_configuration(resource_id=resource_id)
aks_target = ComputeTarget.attach(workspace = ws, 
                                  name = create_name, 
                                  attach_configuration=attach_config)
## Wait for the operation to complete
aks_target.wait_for_provisioning(True)```

### a. Activation App Insights
In order to enable App Insights in your service you will need to update your AKS configuration file:

In [12]:
#Set the web service configuration
aks_deployment_config = AksWebservice.deploy_configuration(enable_app_insights=True)

### b. Déploiement du service

In [14]:
%%time
if aks_target.provisioning_state== "Succeeded": 
    aks_service_name ='aks-ex12'
    aks_service = Model.deploy(ws,
                               aks_service_name, 
                               [model], 
                               inference_config, 
                               aks_deployment_config, 
                               deployment_target = aks_target)    
    aks_service.wait_for_deployment(show_output = True)
    print(aks_service.state)
else:
    raise ValueError("AKS provisioning failed. Error: ", aks_service.error)

Running................
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 173 ms, sys: 20.7 ms, total: 194 ms
Wall time: 1min 33s


> Prévoir 2 minutes de temps de traitements

## 8. Réalisation de plusieurs tests d'appel au modèle de ML exposé dans AKS

In [16]:
%%time

test_sample1 = json.dumps({'data': [
    [1,28,13,45,54,6,57,8,8,10], 
    [101,9,8,37,6,45,4,3,2,41]
]})
test_sample1 = bytes(test_sample1,encoding='utf8')

if aks_service.state == "Healthy":
    prediction1 = aks_service.run(input_data=test_sample1)
    print("Prédictions du modèle : ", prediction1)
    print()
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aks_service.error)

Prédictions du modèle :  [4032.0580991117286, 11618.028587878263]

CPU times: user 1.13 ms, sys: 2.83 ms, total: 3.96 ms
Wall time: 30.2 ms


In [17]:
%%time

test_sample2 = json.dumps({'data': [
    [2,28,13,45,54,6,57,8,8,50], 
    [10,9,8,37,6,45,4,3,2,43]
]})
test_sample2 = bytes(test_sample2,encoding='utf8')

if aks_service.state == "Healthy":
    prediction2 = aks_service.run(input_data=test_sample2)
    print("Prédictions du modèle : ", prediction2)
    print()
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aks_service.error)

Prédictions du modèle :  [7909.812215422598, 11478.00168731773]

CPU times: user 3.28 ms, sys: 0 ns, total: 3.28 ms
Wall time: 13.7 ms


In [21]:
%%time

test_sample3 = json.dumps({'data': [
    [24,48,3,45,54,6,57,8,8,50], 
    [13,5,55,37,6,45,4,3,2,43]
]})
test_sample3 = bytes(test_sample3,encoding='utf8')

if aks_service.state == "Healthy":
    prediction3 = aks_service.run(input_data=test_sample3)
    print("Prédictions du modèle : ", prediction3)
    print()
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aks_service.error)

Prédictions du modèle :  [1107.1856643704434, 31507.059368998056]

CPU times: user 2.37 ms, sys: 1.12 ms, total: 3.49 ms
Wall time: 11.8 ms


## 9. Accés aux données de télémétries AppInsights
Aller dans le portail Azure pour visualiser les informations de télémétries disponibles depuis AKS

<img src="https://github.com/retkowsky/images/blob/master/aksresults.jpg?raw=true">

## 10. Désactivation

### Suppression services AKS ACI et modèle

In [23]:
#aks_service.delete()
#aci_service.delete()
#model.delete()